In [8]:
# -*- coding: utf-8 -*-
"""
Created on Mon Nov  8 13:43:01 2021

@author: luciano 
"""
import numpy as np
from qiskit.utils import QuantumInstance
from qiskit import IBMQ, Aer
from main import *
from qiskit.compiler import transpile
from qiskit.providers.ibmq.managed import IBMQJobManager
from joblib import Parallel, delayed
from time import time

In [2]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic')

backends = ['ibmq_casablanca',
            'ibmq_toronto',
           'ibmq_brooklyn'] 

#Job_id = ['542639f734be450e9ddc3bdffa2f21da-1636392124647574',
#            '9b1053cd3b9d44d79ce8124ac8eec4a9-16363921617092857',
#            'a98c029d0a684003b5aa783f10caaec4-16363922621417603']

Job_id = ['102db23bbe2045698de92ed62ac319a9-16365048283624237']

qasm     = provider.get_backend('ibmq_qasm_simulator')
simulator = Aer.get_backend('aer_simulator')
job_manager = IBMQJobManager()

In [3]:
for j in [0]:
    backend = provider.get_backend(backends[j])
    job_id  = Job_id[j]
    dpmt = device_process_measurement_tomography( backend )
    circuits_dpmt = dpmt.circuits()
    job = job_manager.retrieve_job_set(job_id,provider)

In [4]:
results = job.results().combine_results()

In [6]:
def function_montecarlo(idex):
    
    choi_single, choi_double, gateset  = dpmt.fit( results, paralell=False, gate_set=True, resampling=2**13 )
    parall_qubits = dpmt._parall_qubits
    num_qubits    = dpmt._num_qubits
    quantities = []
    for k in range(num_qubits):
        quantities.append( Quantities( choi_single[k][0], choi_single[k][1] ) )

    quantities_2 = []
    cros_quantities = []
    for i in range(len(parall_qubits)):
        for j in range(len(parall_qubits[i])):
            k = parall_qubits[i][j][0]
            l = parall_qubits[i][j][1]
            cros_quantities.append( Cross_Quantities( choi_single[k][0], choi_single[k][1],
                                                      choi_single[l][0], choi_single[l][1],
                                                      choi_double[i][j][0], choi_double[i][j][1]
                                                    )  )
            quantities_2.append( Quantities( choi_double[i][j][0], choi_double[i][j][1] ) )
        
    np.save( 'datos_mc_{}'.format(idex), np.array([ quantities, quantities_2, cros_quantities ], dtype=object)  )
            
    return [ quantities, quantities_2, cros_quantities  ]

In [10]:
mc = 4
t1 = time()
datos = Parallel(n_jobs=-1)( delayed( function_montecarlo )(j) for j in range(mc) ) 
t2 = time()
print(t2-t1)

558.2008113861084


100